In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.model_selection import train_test_split
import pandas as pd

In [ ]:

# drop passenger id , name and cabin(since i am not able to split test_cabin and train_cabin it into 3 due i.e. cabin , seat, deck)
train=pd.read_csv("/kaggle/input/spaceship-titanic/train.csv").drop(["PassengerId",'Name','Cabin'],axis=1)
#drop all the rows with null values   
train_new=train.dropna(axis=0,how='any')
#one hot encode x
x=pd.get_dummies(train_new.drop('Transported', axis=1))
#one hot encode target labels
y=pd.get_dummies(train_new['Transported'])
x.shape, y.shape

In [ ]:
# lets visualize lables w.r.t to categrical variables
import seaborn as sns
import matplotlib.pyplot as plt
sns.catplot(x='HomePlanet',y='Transported',col='Destination',data=train,hue='CryoSleep',kind='bar')
plt.show()

# We can see that the higher number of individuals in cryosleep made it to their desitnations. HomePlanet and Destination location 
# does not play a major role in deciding the outcome.

In [ ]:
#format the test data same as train data to fit the model for results
test=pd.read_csv("/kaggle/input/spaceship-titanic/test.csv")
new_test=pd.get_dummies(test.drop(["PassengerId",'Name','Cabin'],axis=1))
new_test.head(),new_test.shape

In [ ]:
#It is evident that normalizing the data gives greater accuracy
def min_max_normalized(data):
    col_max = np.max(data, axis=0)
    col_min = np.min(data, axis=0)
    return np.divide(data - col_min, col_max - col_min)

x_train_normal=min_max_normalized(x)
x_test_normal=min_max_normalized(new_test)
x_train_normal.head()

In [ ]:
#callback for stopping training when loss<0.42

class mycallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self,epochs,logs={}):
        if (logs.get('loss')<0.42):
            print("\nStopping training since loss is less than 0.42")
            self.model.stop_training=True
            
            
    

In [ ]:
#set random seed to 42 such that model generate same random input parameters.
tf.random.set_seed(42)

model_21=tf.keras.Sequential([
    tf.keras.layers.Dense(30,activation='relu'),
    tf.keras.layers.Dense(15,activation='relu'),
    tf.keras.layers.Dense(2,activation="softmax")
])

model_21.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
             optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
             metrics=['accuracy'])

history_21=model_21.fit(x_train_normal,y,epochs=100,callbacks=[mycallback()])

In [ ]:
#display the summary of model
model_21.summary()

In [ ]:
#visualize the evaluation metrics of model
pd.DataFrame(history_21.history).plot()
plt.ylabel('loss')
plt.xlabel('epochs')

In [ ]:
#high end view of model
tf.keras.utils.plot_model(model_21)

In [ ]:
#make preictions
predictions=tf.argmax(tf.squeeze(model_21.predict(x_test_normal)),axis=1)

predictions

In [ ]:
l=[]
for i in predictions:
    if i==1:
        l.append(True)
    else:
        l.append(False)
        

In [ ]:
output = pd.DataFrame({'PassengerId': test.PassengerId, 'Transported': l})
output.to_csv('submission.csv', index=False)